In [0]:
import torch
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

  0%|          | 0/26421880 [00:00<?, ?it/s]

26427392it [00:00, 73150694.57it/s]                              


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw


32768it [00:00, 459015.75it/s]
  5%|▍         | 212992/4422102 [00:00<00:02, 1901742.82it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw


4423680it [00:00, 24563920.00it/s]                           
8192it [00:00, 166882.34it/s]


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw
Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw
Processing...
Done!


In [0]:
from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [0]:
model = Classifier()

images, labels = next(iter(testloader))
# Get the class probabilities
ps = torch.exp(model(images))
# Make sure the shape is appropriate, we should get 10 class probabilities for 64 examples
print(ps.shape)

torch.Size([64, 10])


In [0]:
top_p,top_class=ps.topk(1,dim=1)

In [0]:
print(top_class[:10,:])

tensor([[2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2]])


In [0]:
equals = top_class == labels
equals.shape

torch.Size([64, 64])

In [0]:
equals = top_class == labels.view(*top_class.shape)
equals.shape
# equals

torch.Size([64, 1])

In [0]:
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')

Accuracy: 7.8125%


In [0]:
labels

tensor([5, 5, 5, 9, 4, 3, 7, 4, 9, 7, 2, 1, 3, 4, 5, 3, 7, 7, 9, 6, 9, 7, 8, 8,
        9, 8, 8, 2, 3, 3, 5, 5, 5, 4, 6, 8, 7, 8, 6, 7, 0, 1, 5, 6, 1, 2, 4, 9,
        7, 0, 4, 6, 2, 0, 5, 1, 0, 9, 2, 5, 5, 7, 9, 7])

In [0]:


model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 30
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            for images, labels in testloader:
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))



Epoch: 1/30..  Training Loss: 0.517..  Test Loss: 0.423..  Test Accuracy: 0.851
Epoch: 2/30..  Training Loss: 0.393..  Test Loss: 0.407..  Test Accuracy: 0.854
Epoch: 3/30..  Training Loss: 0.355..  Test Loss: 0.413..  Test Accuracy: 0.857
Epoch: 4/30..  Training Loss: 0.334..  Test Loss: 0.398..  Test Accuracy: 0.855
Epoch: 5/30..  Training Loss: 0.315..  Test Loss: 0.395..  Test Accuracy: 0.859
Epoch: 6/30..  Training Loss: 0.306..  Test Loss: 0.363..  Test Accuracy: 0.872
Epoch: 7/30..  Training Loss: 0.292..  Test Loss: 0.366..  Test Accuracy: 0.876
Epoch: 8/30..  Training Loss: 0.285..  Test Loss: 0.395..  Test Accuracy: 0.867
Epoch: 9/30..  Training Loss: 0.274..  Test Loss: 0.355..  Test Accuracy: 0.875
Epoch: 10/30..  Training Loss: 0.266..  Test Loss: 0.367..  Test Accuracy: 0.879


In [0]:
class Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1=nn.linear(784,256)
    self.fc2=nn.linear(256,128)
    self.fc3=nn.linear(128,64)
    self.fc4=nn.linear(64,10)

    self.dropout=nn.Dropout(p=0.2)

  def forward(self,x):
    x=x.view(x.shape[0],-1)  #(60,784)

    x=self.dropout(F.relu(self.fc1(x)))
    x=self.dropout(F.relu(self.fc2(x)))
    x=self.dropout(F.relu(self.fc3(x)))

    x=F.log_softmax(self.fc4(x),dim=1)

    return x


In [0]:
model=Classifier()
criterion=nn.NLLLoss()
optimizer=optim.Adam(model.parameters(),lr=0.003)

epochs=30
steps=0

